# vcfstats
# What does this do?

### Run rtg-tools vcfstats
- Get list of all variants.gvcf.gz files
- Get list of all rtg_vcfstats.txt files
- Convert each to lists of sample IDs
- Compare lists to find samples that are missing rtg_vcfstats.txt files
- Use make-batch-tsv-from-input-sample.py to generate dsub task file
- Run dsub to generate missing rtg_vcfstats.txt files

### Convert vcfstats result files to CSV format
- Get list of all rtg_vcfstats.txt files
- Get list of all rtg_vcfstats.txt.csv files
- 

### Concatenate CSV Files

# Code

## 0. Check that environment variables have been loaded correctly

Environment variables are imported from the mvp-profile.sh file. If this echo command does not return anything, try sourcing it from the command console. If any of the values are incorrect, change them in mvp-profile.sh, save it, and source it again.

In [ ]:
# source mvp_profile.sh
echo "Date stamp: ${date_stamp}"
echo "Home directory: ${mvp_hub}"
echo "Project: ${mvp_project}"
echo "Bucket: ${mvp_bucket}"
echo "Zone: ${mvp_zone}"

## 1. Run rtg-tools vcfstats

#### Create file accounting directory it doesn't already exist

In [ ]:
mkdir -p ${mvp_hub}/vcfstats/file-accounting/${date_stamp}

#### Get list of sample IDs for gvcf files that already exist on Google Cloud Storage

In [ ]:
accounting_dir="${mvp_hub}/vcfstats/file-accounting/${date_stamp}"

gsutil ls gs://${mvp_bucket}/data/bina-deliverables/*/*/VariantCalling/variants.gvcf.gz \
    > ${accounting_dir}/gs-bina-gvcf-${date_stamp}.txt
cut -d'/' -f6 ${mvp_hub}/vcfstats/file-accounting/${date_stamp}/gs-bina-gvcf-${date_stamp}.txt \
    > ${accounting_dir}/gs-bina-gvcf-sample-ids-${date_stamp}.txt

#### Get list sample IDs for vcfstats files that already exist on Google Cloud Storage

In [ ]:
accounting_dir="${mvp_hub}/vcfstats/file-accounting/${date_stamp}"

gsutil ls gs://${mvp_bucket}/dsub/vcfstats/rtg-tools/objects/*_rtg_vcfstats.txt \
    > ${accounting_dir}/gs-bina-vcfstats-rtg-${date_stamp}.txt
cut -d '/' -f8 ${accounting_dir}/gs-bina-vcfstats-rtg-${date_stamp}.txt \
    | cut -d'_' -f1 > ${accounting_dir}/gs-bina-vcfstats-rtg-sample-ids-${date_stamp}.txt

#### Get difference between lists of sample IDs to find out which samples are missing vcfstats files

In [1]:
accounting_dir="${mvp_hub}/vcfstats/file-accounting/${date_stamp}"

diff --new-line-format="" --unchanged-line-format "" \
    <(sort ${accounting_dir}/gs-bina-gvcf-sample-ids-${date_stamp}.txt) \
    <(sort ${accounting_dir}/gs-bina-vcfstats-rtg-sample-ids-${date_stamp}.txt) \
    > ${accounting_dir}/gs-bina-vcfstats-rtg-missing-sample-ids-${date_stamp}.txt
grep -F -f ${accounting_dir}/gs-bina-vcfstats-rtg-missing-sample-ids-${date_stamp}.txt \
    ${accounting_dir}/gs-bina-gvcf-${date_stamp} \
    > ${accounting_dir}/gs-bina-vcfstats-rtg-missing-${date_stamp}.txt

: 1

#### Create dsub TSV task file to generate missing vcfstats files

In [ ]:
${mvp_hub}/bin/make-batch-tsv-from-input-sample.py \
    -i ${mvp_hub}/vcfstats/file_accounting/${date_stamp}/gs-bina-vcfstats-rtg-missing-${date_stamp}.txt \
    -t ${mvp_hub}/vcfstats/dsub-inputs/rtg-tools/gs-bina-vcfstats-rtg-missing-${date_stamp}.tsv \
    -o gs://${mvp_bucket}/dsub/vcfstats/rtg-tools/objects \
    -s rtg_vcfstats.txt

#### Run RTG vcfstats dsub tasks

In [ ]:
dsub \
    --zones "${mvp_region}" \
    --project ${mvp_project} \
    --logging gs://${mvp_bucket}/dsub/vcfstats/rtg-tools/logs/${date_stamp} \
    --image gcr.io/${mvp_project}/rtg-tools:1.0 \
    --command 'rtg vcfstats ${INPUT} > ${OUTPUT}' \
    --tasks ${mvp_hub}/vcfstats/dsub-inputs/rtg-tools/gs-bina-vcfstats-rtg-missing-${date_stamp}.tsv ${dsub_range} \
    #--dry-run

## 2. Convert vcfstats result files to CSV format

#### Get list of sample IDs for vcfstats files that already exist on Google Cloud Storage

In [2]:
accounting_dir="${mvp_hub}/vcfstats/file-accounting/${date_stamp}"

gsutil ls gs://${mvp_bucket}/dsub/vcfstats/rtg-tools/objects/*_rtg_vcfstats.txt \
    > ${accounting_dir}/gs-bina-vcfstats-rtg-${date_stamp}.txt
cut -d'/' -f8 ${accounting_dir}/gs-bina-vcfstats-rtg-${date_stamp}.txt \
    | cut -d'_' -f1 > ${accounting_dir}/gs-bina-vcfstats-rtg-sample-ids-${date_stamp}.txt

#### Get list of sample IDs for vcfstats CSV files that already exist on Google Cloud Storage

In [ ]:
accounting_dir="${mvp_hub}/vcfstats/file-accounting/${date_stamp}"

gsutil ls gs://${mvp_bucket}/dsub/vcfstats/text-to-table/objects/*_rtg_vcfstats.txt.csv \
    > ${accounting_dir}/gs-bina-vcfstats-csv-${date_stamp}.txt
cut -d'/' -f8 ${accounting_dir}/gs-bina-vcfstats-csv-${date_stamp}.txt \
    | cut -d'_' -f1 > ${accounting_dir}/gs-bina-vcfstats-csv-sample-ids-${date_stamp}.txt

#### Get difference between lists of sample IDs to find out which samples are missing vcfstats files

In [ ]:
accounting_dir="${mvp_hub}/vcfstats/file-accounting/${date_stamp}"

diff \
    --new-line-format="" \
    --unchanged-line-format "" \
    <(sort ${accounting_dir}/gs-bina-vcfstats-rtg-sample-ids-${date_stamp}.txt) \
    <(sort ${accounting_dir}/gs-bina-vcfstats-csv-sample-ids-${date_stamp}.txt) \
    > ${accounting_dir}/gs-bina-vcfstats-csv-sample-ids-missing-${date_stamp}.txt
grep -F -f \
    ${accounting_dir}/gs-bina-vcfstats-csv-sample-ids-missing-${date_stamp}.txt \
    ${accounting_dir}/gs-bina-vcfstats-rtg-${date_stamp}.txt \
    > ${accounting_dir}/gs-bina-vcfstats-csv-missing-${date_stamp}.txt

#### Convert file list to dsub TSV files

In [ ]:
${mvp_hub}/bin/make-batch-tsv-from-input-file.py \
    -i ${mvp_hub}/vcfstats/file-accounting/${date_stamp}/gs-bina-vcfstats-csv-missing-${date_stamp}.txt \
    -t ${mvp_hub}/vcfstats/dsub-inputs/text-to-table/gs-bina-vcfstats-csv-missing-${date_stamp}.tsv \
    -o gs://${mvp_bucket}/dsub/vcfstats/text-to-table/objects \
    -s csv \
    -c rtg_vcfstats \
    -e rtg-vcfstats-${date_stamp}

#### Run dsub task

In [ ]:
dsub \
    --zones ${mvp_region} \
    --project ${mvp_project} \
    --logging gs://${mvp_bucket}/dsub/vcfstats/text-to-table/logs/${date_stamp} \
    --image gcr.io/gbsc-gcp-project-mvp/text-to-table:0.2.0 \
    --command 'text2table -s ${SCHEMA} -o ${OUTPUT} -v series=${SERIES},sample=${SAMPLE_ID} ${INPUT}' \
    --tasks ${mvp_hub}/vcfstats/dsub-inputs/text-to-table/gs-bina-vcfstats-csv-missing-${date_stamp}.tsv ${dsub_range} \
    #--dry-run

## 3. Concatenate CSV Files

#### Get new list of completed results files

In [ ]:
accounting_dir="${mvp_hub}/vcfstats/file-accounting/${date_stamp}"

gsutil ls gs://${mvp_bucket}/dsub/vcfstats/text-to-table/objects/*_rtg_vcfstats.txt.csv \
    > ${accounting_dir}/gs-bina-vcfstats-csv-${date_stamp}.txt

#### Run dsub task

In [ ]:
dsub \
    --zones ${mvp_region} \
    --project ${mvp_project} \
    --logging gs://${mvp_bucket}/dsub/vcfstats/concat/logs/${date_stamp} \
    --image gcr.io/gbsc-gcp-project-mvp/text-to-table:0.2.0 \
    --disk-size 100 \
    --script ${mvp_hub}/vcfstats/dsub-scripts/text-to-table.sh \
    --input INPUT_FILES=gs://${mvp_bucket}/dsub/vcfstats/text-to-table/objects/*_vcfstats.txt.csv \
    --output CONCAT_FILE=gs://${mvp_bucket}/dsub/vcfstats/concat/concat_vcfstats.txt.csv \
    --env SCHEMA=rtg_vcfstats \
    --env SERIES=vcfstats-${date_stamp}